In [25]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from itertools import combinations
from scipy.stats import ttest_ind

### Описание датасета

Большая часть данный решена при помощи SQL-запросов, необходимо провести небольшой анализ.

Цель — оптимизировать полеты зимнего периода.

* flight_id					                -- Идентификатор рейса       
* flight_no					                -- Номер рейса      
* scheduled_departure 			            -- Запланированное время отправления
* scheduled_arrival			             	-- Запланированное время прибытия
* departure_airport				            -- Аэропорт отправления 
* departure_airprts.city AS departure_city	-- Город отправления 
* arrival_airport				            -- Аэропорт прибытия
* arrival_airprts.city AS arrival_city		-- Город прибытия
* actual_departure				            -- Фактическое время отправления 
* actual_arrival				            -- Фактическое время отправления
* departure_delay				            -- Время задержки отправления
* arrival_delay					            -- Время задержки прибытия  
* sum_price_tiket			               	-- Сумма билетов 
* flight_time					            -- Фактическое время в пути 
* tons_of_fuel_per_hour			            -- Расход топлива (тонн/час)  
* price_per_tons_fuel_per_month,	     	-- Цена тонны топлива в месяц вылета	
* сosts_per_hour 				            -- Расход в денежном выражении в час
* flight_profit 				            -- Прибыль рейса («Сумма билетов» – «время полёта» * «расход топлива» * «цену топлива»)

In [26]:
import warnings
from pylab import rcParams
%config InlineBackend.figure_format = 'svg'
rcParams['figure.figsize'] = 12, 6

warnings.simplefilter('ignore')
%matplotlib inline

In [27]:
df = pd.read_csv('Dataset.csv')

In [28]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 127 entries, 0 to 126
Data columns (total 18 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   flight_id                      127 non-null    int64  
 1   flight_no                      127 non-null    object 
 2   scheduled_departure            127 non-null    object 
 3   scheduled_arrival              127 non-null    object 
 4   departure_airport              127 non-null    object 
 5   departure_city                 127 non-null    object 
 6   arrival_airport                127 non-null    object 
 7   arrival_city                   127 non-null    object 
 8   actual_departure               127 non-null    object 
 9   actual_arrival                 127 non-null    object 
 10  departure_delay                127 non-null    float64
 11  arrival_delay                  127 non-null    float64
 12  sum_price_tiket                118 non-null    flo

In [29]:
df

,flight_id,flight_no,scheduled_departure,scheduled_arrival,departure_airport,departure_city,arrival_airport,arrival_city,actual_departure,actual_arrival,departure_delay,arrival_delay,sum_price_tiket,flight_time,tons_of_fuel_per_hour,price_per_tons_fuel_per_month,сosts_per_hour,profit_flight
0,136119,PG0252,2017-01-03T10:05:00Z,2017-01-03T11:45:00Z,AAQ,Anapa,SVO,Moscow,2017-01-03T10:08:00Z,2017-01-03T11:49:00Z,3.0,4.0,1653000.0,1.683333,2.6,48893,213988.363333,1.439012e+06
1,136120,PG0252,2017-02-24T10:05:00Z,2017-02-24T11:45:00Z,AAQ,Anapa,SVO,Moscow,2017-02-24T10:07:00Z,2017-02-24T11:46:00Z,2.0,1.0,1605400.0,1.650000,2.6,46672,200222.880000,1.405177e+06
2,136122,PG0252,2017-01-08T10:05:00Z,2017-01-08T11:45:00Z,AAQ,Anapa,SVO,Moscow,2017-01-08T10:08:00Z,2017-01-08T11:48:00Z,3.0,3.0,1431000.0,1.666667,2.6,48893,211869.666667,1.219130e+06
3,136130,PG0252,2017-02-13T10:05:00Z,2017-02-13T11:45:00Z,AAQ,Anapa,SVO,Moscow,2017-02-13T10:07:00Z,2017-02-13T11:46:00Z,2.0,1.0,1556600.0,1.650000,2.6,46672,200222.880000,1.356377e+06
4,136131,PG0252,2017-01-28T10:05:00Z,2017-01-28T11:45:00Z,AAQ,Anapa,SVO,Moscow,2017-01-28T10:07:00Z,2017-01-28T11:46:00Z,2.0,1.0,1812800.0,1.650000,2.6,48893,209750.970000,1.603049e+06
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
122,136936,PG0480,2017-01-02T09:25:00Z,2017-01-02T10:15:00Z,AAQ,Anapa,EGO,Belgorod,2017-01-02T09:28:00Z,2017-01-02T10:18:00Z,3.0,3.0,632400.0,0.833333,1.7,48893,69265.083333,5.631349e+05
123,136937,PG0480,2017-01-03T09:25:00Z,2017-01-03T10:15:00Z,AAQ,Anapa,EGO,Belgorod,2017-01-03T09:29:00Z,2017-01-03T10:19:00Z,4.0,4.0,626100.0,0.833333,1.7,48893,69265.083333,5.568349e+05
124,136951,PG0480,2017-02-02T09:25:00Z,2017-02-02T10:15:00Z,AAQ,Anapa,EGO,Belgorod,2017-02-02T09:28:00Z,2017-02-02T10:17:00Z,3.0,2.0,720600.0,0.816667,1.7,46672,64796.293333,6.558037e+05
125,136953,PG0480,2017-02-27T09:25:00Z,2017-02-27T10:15:00Z,AAQ,Anapa,EGO,Belgorod,2017-02-27T09:27:00Z,2017-02-27T10:17:00Z,2.0,2.0,765300.0,0.833333,1.7,46672,66118.666667,6.991813e+05


### Обработка пропусков

Заполним пустые ячейки None

In [30]:
df.sum_price_tiket.replace('', np.nan, inplace = True)

In [31]:
df.sum_price_tiket = df.sum_price_tiket.astype(str).apply(lambda x: 0 if x.strip() == '' else x)

In [32]:
df.loc[df.sum_price_tiket == 'nan']

,flight_id,flight_no,scheduled_departure,scheduled_arrival,departure_airport,departure_city,arrival_airport,arrival_city,actual_departure,actual_arrival,departure_delay,arrival_delay,sum_price_tiket,flight_time,tons_of_fuel_per_hour,price_per_tons_fuel_per_month,сosts_per_hour,profit_flight
59,136511,PG0194,2017-02-21T06:10:00Z,2017-02-21T11:15:00Z,AAQ,Anapa,NOZ,Novokuznetsk,2017-02-21T06:15:00Z,2017-02-21T11:25:00Z,5.0,10.0,nan,5.166667,2.6,46672,626960.533333,-626960.533333
60,136513,PG0194,2017-02-28T06:10:00Z,2017-02-28T11:15:00Z,AAQ,Anapa,NOZ,Novokuznetsk,2017-02-28T06:13:00Z,2017-02-28T11:23:00Z,3.0,8.0,nan,5.166667,2.6,46672,626960.533333,-626960.533333
61,136514,PG0194,2017-02-07T06:10:00Z,2017-02-07T11:15:00Z,AAQ,Anapa,NOZ,Novokuznetsk,2017-02-07T06:14:00Z,2017-02-07T11:13:00Z,4.0,-2.0,nan,4.983333,2.6,46672,604713.546667,-604713.546667
62,136523,PG0194,2017-01-10T06:10:00Z,2017-01-10T11:15:00Z,AAQ,Anapa,NOZ,Novokuznetsk,2017-01-10T06:14:00Z,2017-01-10T11:26:00Z,4.0,11.0,nan,5.200000,2.6,48893,661033.360000,-661033.360000
63,136540,PG0194,2017-01-17T06:10:00Z,2017-01-17T11:15:00Z,AAQ,Anapa,NOZ,Novokuznetsk,2017-01-17T06:14:00Z,2017-01-17T11:21:00Z,4.0,6.0,nan,5.116667,2.6,48893,650439.876667,-650439.876667
64,136544,PG0194,2017-01-31T06:10:00Z,2017-01-31T11:15:00Z,AAQ,Anapa,NOZ,Novokuznetsk,2017-01-31T06:10:00Z,2017-01-31T11:14:00Z,0.0,-1.0,nan,5.066667,2.6,48893,644083.786667,-644083.786667
65,136546,PG0194,2017-01-03T06:10:00Z,2017-01-03T11:15:00Z,AAQ,Anapa,NOZ,Novokuznetsk,2017-01-03T06:13:00Z,2017-01-03T11:15:00Z,3.0,0.0,nan,5.033333,2.6,48893,639846.393333,-639846.393333
66,136560,PG0194,2017-01-24T06:10:00Z,2017-01-24T11:15:00Z,AAQ,Anapa,NOZ,Novokuznetsk,2017-01-24T06:13:00Z,2017-01-24T11:18:00Z,3.0,3.0,nan,5.083333,2.6,48893,646202.483333,-646202.483333
67,136567,PG0194,2017-02-14T06:10:00Z,2017-02-14T11:15:00Z,AAQ,Anapa,NOZ,Novokuznetsk,2017-02-14T06:12:00Z,2017-02-14T11:23:00Z,2.0,8.0,nan,5.183333,2.6,46672,628982.986667,-628982.986667


In [33]:
df.loc[df.profit_flight <0]

,flight_id,flight_no,scheduled_departure,scheduled_arrival,departure_airport,departure_city,arrival_airport,arrival_city,actual_departure,actual_arrival,departure_delay,arrival_delay,sum_price_tiket,flight_time,tons_of_fuel_per_hour,price_per_tons_fuel_per_month,сosts_per_hour,profit_flight
59,136511,PG0194,2017-02-21T06:10:00Z,2017-02-21T11:15:00Z,AAQ,Anapa,NOZ,Novokuznetsk,2017-02-21T06:15:00Z,2017-02-21T11:25:00Z,5.0,10.0,nan,5.166667,2.6,46672,626960.533333,-626960.533333
60,136513,PG0194,2017-02-28T06:10:00Z,2017-02-28T11:15:00Z,AAQ,Anapa,NOZ,Novokuznetsk,2017-02-28T06:13:00Z,2017-02-28T11:23:00Z,3.0,8.0,nan,5.166667,2.6,46672,626960.533333,-626960.533333
61,136514,PG0194,2017-02-07T06:10:00Z,2017-02-07T11:15:00Z,AAQ,Anapa,NOZ,Novokuznetsk,2017-02-07T06:14:00Z,2017-02-07T11:13:00Z,4.0,-2.0,nan,4.983333,2.6,46672,604713.546667,-604713.546667
62,136523,PG0194,2017-01-10T06:10:00Z,2017-01-10T11:15:00Z,AAQ,Anapa,NOZ,Novokuznetsk,2017-01-10T06:14:00Z,2017-01-10T11:26:00Z,4.0,11.0,nan,5.200000,2.6,48893,661033.360000,-661033.360000
63,136540,PG0194,2017-01-17T06:10:00Z,2017-01-17T11:15:00Z,AAQ,Anapa,NOZ,Novokuznetsk,2017-01-17T06:14:00Z,2017-01-17T11:21:00Z,4.0,6.0,nan,5.116667,2.6,48893,650439.876667,-650439.876667
64,136544,PG0194,2017-01-31T06:10:00Z,2017-01-31T11:15:00Z,AAQ,Anapa,NOZ,Novokuznetsk,2017-01-31T06:10:00Z,2017-01-31T11:14:00Z,0.0,-1.0,nan,5.066667,2.6,48893,644083.786667,-644083.786667
65,136546,PG0194,2017-01-03T06:10:00Z,2017-01-03T11:15:00Z,AAQ,Anapa,NOZ,Novokuznetsk,2017-01-03T06:13:00Z,2017-01-03T11:15:00Z,3.0,0.0,nan,5.033333,2.6,48893,639846.393333,-639846.393333
66,136560,PG0194,2017-01-24T06:10:00Z,2017-01-24T11:15:00Z,AAQ,Anapa,NOZ,Novokuznetsk,2017-01-24T06:13:00Z,2017-01-24T11:18:00Z,3.0,3.0,nan,5.083333,2.6,48893,646202.483333,-646202.483333
67,136567,PG0194,2017-02-14T06:10:00Z,2017-02-14T11:15:00Z,AAQ,Anapa,NOZ,Novokuznetsk,2017-02-14T06:12:00Z,2017-02-14T11:23:00Z,2.0,8.0,nan,5.183333,2.6,46672,628982.986667,-628982.986667


9 пустых ячеек в интересующих нас колонках относятся к рейсам из Анапы в Новокузнецк.\
Для дальнейшего аналаза удалим их из датасета.

In [34]:
y = df.flight_id[df.profit_flight <0].values.tolist()
y

[136511, 136513, 136514, 136523, 136540, 136544, 136546, 136560, 136567]

In [35]:
df_1 = df.copy()

Удалим убыточные рейсы, для подсчета средней стоимости, оставшихся рейсов.

In [36]:
df_1 = df_1.drop(df_1.loc[df_1.profit_flight <0].index)

In [37]:
df_1

,flight_id,flight_no,scheduled_departure,scheduled_arrival,departure_airport,departure_city,arrival_airport,arrival_city,actual_departure,actual_arrival,departure_delay,arrival_delay,sum_price_tiket,flight_time,tons_of_fuel_per_hour,price_per_tons_fuel_per_month,сosts_per_hour,profit_flight
0,136119,PG0252,2017-01-03T10:05:00Z,2017-01-03T11:45:00Z,AAQ,Anapa,SVO,Moscow,2017-01-03T10:08:00Z,2017-01-03T11:49:00Z,3.0,4.0,1653000.0,1.683333,2.6,48893,213988.363333,1.439012e+06
1,136120,PG0252,2017-02-24T10:05:00Z,2017-02-24T11:45:00Z,AAQ,Anapa,SVO,Moscow,2017-02-24T10:07:00Z,2017-02-24T11:46:00Z,2.0,1.0,1605400.0,1.650000,2.6,46672,200222.880000,1.405177e+06
2,136122,PG0252,2017-01-08T10:05:00Z,2017-01-08T11:45:00Z,AAQ,Anapa,SVO,Moscow,2017-01-08T10:08:00Z,2017-01-08T11:48:00Z,3.0,3.0,1431000.0,1.666667,2.6,48893,211869.666667,1.219130e+06
3,136130,PG0252,2017-02-13T10:05:00Z,2017-02-13T11:45:00Z,AAQ,Anapa,SVO,Moscow,2017-02-13T10:07:00Z,2017-02-13T11:46:00Z,2.0,1.0,1556600.0,1.650000,2.6,46672,200222.880000,1.356377e+06
4,136131,PG0252,2017-01-28T10:05:00Z,2017-01-28T11:45:00Z,AAQ,Anapa,SVO,Moscow,2017-01-28T10:07:00Z,2017-01-28T11:46:00Z,2.0,1.0,1812800.0,1.650000,2.6,48893,209750.970000,1.603049e+06
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
122,136936,PG0480,2017-01-02T09:25:00Z,2017-01-02T10:15:00Z,AAQ,Anapa,EGO,Belgorod,2017-01-02T09:28:00Z,2017-01-02T10:18:00Z,3.0,3.0,632400.0,0.833333,1.7,48893,69265.083333,5.631349e+05
123,136937,PG0480,2017-01-03T09:25:00Z,2017-01-03T10:15:00Z,AAQ,Anapa,EGO,Belgorod,2017-01-03T09:29:00Z,2017-01-03T10:19:00Z,4.0,4.0,626100.0,0.833333,1.7,48893,69265.083333,5.568349e+05
124,136951,PG0480,2017-02-02T09:25:00Z,2017-02-02T10:15:00Z,AAQ,Anapa,EGO,Belgorod,2017-02-02T09:28:00Z,2017-02-02T10:17:00Z,3.0,2.0,720600.0,0.816667,1.7,46672,64796.293333,6.558037e+05
125,136953,PG0480,2017-02-27T09:25:00Z,2017-02-27T10:15:00Z,AAQ,Anapa,EGO,Belgorod,2017-02-27T09:27:00Z,2017-02-27T10:17:00Z,2.0,2.0,765300.0,0.833333,1.7,46672,66118.666667,6.991813e+05


In [38]:
profit_mean = round(df_1.profit_flight.mean(),2)
profit_median = round(df_1.profit_flight.median(),2)
profit_mode = df_1.profit_flight.mode()[0]

print('Среднее:',profit_mean)
print('Медиана:',profit_median)
print('Мода:',profit_mode)

Среднее: 1039090.01
Медиана: 942655.83
Мода: 697858.96


### Как рекомендация, можно сделать не прямой рейс Анапы в Новокузнецк, а через Сочи или Краснодар.